# In progress -  The Battle of Neighborhoods

## Week 1

### Presentation of Problem, project approach, and data/map retrieval

#### Pre-project: selecting a city to analysis

After exploring North-America cities; I would like to explore data in Europe. 
As pre-project preparation I browsed Foursquare reviews about the city of Paris (when I am from) and found out that data was not up to date (200 reviews for the Tour Eiffel, back to 2018...). So let's go to London which seems to have a larger Foursquare users base.


#### Deciding where to open a new venue in London (outside Inner City Limits)

London is a great city. It is diverse, multicultural and full of opportunities. But there is also another side to its popularity and appeal. London is an expensive place to live and thrive. Many businesses want to open a venue here and many pay top price to have their window on Piccadilly or Oxford Street. But what about those, who want to start their own business and cannot really afford to open in the City yet? Where is it best to open a new place? Where will it be cheapest and will have enough people living around to be popular? Where the competition is not too overwhelming?

London, capital of United-Kingdom is a large city; actually much larger than Toronto but smaller than New-York. It shares with them similar characteristics: London is multicultural, modern, full of leisure opportunities  and is an attractive business market place.
Like in all big cities, it is very very expensive to run a venue in the most popular Boroughs; so venues owners need to have full understanding of business opportunities before lannching/investing in their companies. Like the famous UK TV show "Location, location, location!", location is the most important criteria for any property and therefore business investment. So we will use data, maps and data science tools to analysis where is(are) the best place (s) to have a venue in London.

We will answer all questions below and we will use next plan:


Week1
1. Retrieve data about London Boroughs, area and population using web page scrapping
2. Filter boroughs to get the extract best ones in depth analysis
3. Create Initial foliam map centred on London


Week2
4. Connect to Foursquare API to obtain data about most popular venues
5. Find optimised number of clusters
6. Visualization with Chloropleth Map
7. Analysis and discussion
8. Conclusion


Retrieve data on London Boroughs using web page scrapping on wikepedia London page

In [3]:
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors


url='https://en.wikipedia.org/wiki/List_of_London_boroughs'

LDF=pd.read_html(url, header=0)[0]

LDF.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E,25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


Clean dataframe: Drop columsn not used; rename some boroughs because of bad qualiy data from wiki page (note 1). 

In [5]:
LF = LDF.drop(['Status','Local authority','Political control','Headquarters','Nr. in map'], axis=1)
LF['Inner'].replace(np.nan,'0', inplace=True)
LF['Borough'].replace('Barking and Dagenham [note 1]','Barking and Dagenham', inplace=True)
LF['Borough'].replace('Greenwich [note 2]','Greenwich', inplace=True)
LF['Borough'].replace('Hammersmith and Fulham [note 4]','Hammersmith and Fulham', inplace=True)
Inn = ['Camden','Greenwich','Hackney','Hammersmith and Fulham','Islington','Kensington and Chelsea','Lewisham','Lambeth','Southwark','Tower Hamlets','Wandsworth','Westminster']
LF.head()
LF['Inner'] = '0'
LF.head()

,Borough,Inner,Area (sq mi),Population (2013 est)[1],Co-ordinates
0,Barking and Dagenham,0,13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E
1,Barnet,0,33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W
2,Bexley,0,23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E
3,Brent,0,16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W
4,Bromley,0,57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E


Because of huge difference in rent price, as well as the ammount of venues in London, we will filter to have only the outer boroughs going forward


In [8]:
LF['Inner'] = LF.Borough.isin(Inn).astype(int)
Out = LF[LF.Inner == 0]
Out = Out.drop(['Inner'], axis=1)
df = Out.rename(columns = {"Area (sq mi)": "Area", 
                            "Population (2013 est)[1]":"Population"})
geolocator = Nominatim(user_agent="London_explorer")
df['Co-ordinates']= df['Borough'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df[['Latitude', 'Longitude']] = df['Co-ordinates'].apply(pd.Series)
df.head()

,Borough,Area,Population,Co-ordinates,Latitude,Longitude
0,Barking and Dagenham,13.93,194352,"(51.5541171, 0.15050434261994267)",51.554117,0.150504
1,Barnet,33.49,369088,"(51.65309, -0.2002261)",51.653090,-0.200226
2,Bexley,23.38,236687,"(-43.5110351, 172.7183763)",-43.511035,172.718376
3,Brent,16.70,317264,"(32.9373463, -87.1647184)",32.937346,-87.164718
4,Bromley,57.97,317899,"(51.4028046, 0.0148142)",51.402805,0.014814


We need to edit the coordinates, that are incorrect. We have to filter the set even more - because rent price is a big issue for the new business, we will get the boroughs with the lowest ones.

In [9]:
Max_Rent = ['102.25','150.75','97','150.75','118.5','129.25','140','102.25','107.75','140','86','161.5','161.5','140','123.75','134.5','118.5','140','129.25','145.25']
df['Max_Rent'] = Max_Rent

df["Max_Rent"] = pd.to_numeric(df["Max_Rent"])
fin = df[df.Max_Rent <= 125]
fin

Long_list = fin['Longitude'].tolist()
Lat_list = fin['Latitude'].tolist()
print ("Old latitude list: ", Lat_list)
print ("Old Longitude list: ", Long_list)

replace_longitudes = {-106.6621329:0.0799, -2.8417544: 0.1837}
replace_latitudes = {50.7164496:51.6636, 51.0358628: 51.5499}

longtitudes_new = [replace_longitudes.get(n7,n7) for n7 in Long_list]
latitudes_new = [replace_latitudes.get(n7,n7) for n7 in Lat_list]

fin = fin.drop(['Co-ordinates', 'Longitude'], axis=1)

fin['Longitude'] = longtitudes_new
fin['Latitude'] = latitudes_new
fin

Old latitude list:  [51.5541171, -43.5110351, 51.4028046, 51.6520851, 51.587929849999995, 51.0043613, 51.41086985, 51.5763203]
Old Longitude list:  [0.15050434261994267, 172.7183763, 0.0148142, -0.0810175, -0.10541010599099046, -2.337474942629507, -0.18809708858824303, 0.0454097]


,Borough,Area,Population,Latitude,Max_Rent,Longitude
0,Barking and Dagenham,13.93,194352,51.554117,102.25,0.150504
2,Bexley,23.38,236687,-43.511035,97.00,172.718376
4,Bromley,57.97,317899,51.402805,118.50,0.014814
8,Enfield,31.74,320524,51.652085,102.25,-0.081018
12,Haringey,11.42,263386,51.587930,107.75,-0.105410
14,Havering,43.35,242080,51.004361,86.00,-2.337475
22,Merton,14.52,203223,51.410870,123.75,-0.188097
24,Redbridge,21.78,288272,51.576320,118.50,0.045410


In [14]:
print("We will make in depth analysis on", fin.shape[0],"London Boroughs")

We will make in depth analysis on 8 London Boroughs


We will now make a London map.
Fistly we need to find London geo coordinates

In [15]:
address = 'London'

geolocator = Nominatim(user_agent="London_explorer")
location = geolocator.geocode(address)
London_latitude = location.latitude
London_longitude = location.longitude
print('The geograpical coordinates of London are {}, {}.'.format(London_latitude, London_longitude))

The geograpical coordinates of London are 51.5073219, -0.1276474.


Create London map using Folium

In [16]:
# import Folium map rendering librairy
import folium

Fin_Brgh = folium.Map(location=[London_latitude, London_longitude], zoom_start=12)


for lat, lng, label in zip(fin['Latitude'], fin['Longitude'], 
                            fin['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color='Red',
        fill=True,
        fill_color='#Blue',
        fill_opacity=0.7).add_to(Fin_Brgh)
Fin_Brgh